Copyright (c) 2023 Habana Labs, Ltd. an Intel Company.

#### Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Using Paramater Efficient Fine Tuning on Llama 2 with 70B Parameters
This example will Fine Tune the Llama2-70B model using Parameter Efficient Fine Tuining (PEFT) and then run inference on a text prompt.  This will be using the Llama2 model with two task examples from the Optimum Habana library on the Hugging Face model repository.   The Optimum Habana library is optimized for Deep Learning training and inference on First-gen Gaudi and Gaudi2 and offers tasks such as text generation, language modeling, question answering and more. For all the examples and models, please refer to the [Optimum Habana GitHub](https://github.com/huggingface/optimum-habana#validated-models).

This example will Fine Tune the Llama2-70B model using Parameter Efficient Fine Tuining (PEFT) on the timdettmers/openassistant-guanaco dataset using the Language-Modeling Task in Optimum Habana.

### Parameter Efficient Fine Tuning with Low Rank Adaptation
Parameter Efficient Fine Tuning is a strategy for adapting large pre-trained language models to specific tasks while minimizing computational and memory demands.   It aims to reduce the computational cost and memory requirements associated with fine-tuning large models while maintaining or even improving their performance.  It does so by adding a smaller task-specific layer, leveraging knowledge distillation, and often relying on few-shot learning, resulting in efficient yet effective models for various natural language understanding tasks.   PEFT starts with a pre-trained language model that has already learned a wide range of language understanding tasks from a large corpus of text data. These models are usually large and computationally expensive.   Instead of fine-tuning the entire pre-trained model, PEFT adds a task-specific layer or a few task-specific layers on top of the pre-trained model. These additional layers are relatively smaller and have fewer parameters compared to the base model.


In [2]:
%cd /root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference

/root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Model Setup: 

##### Install the latest version of the Habana Deepspeed Library

In [3]:
!pip install -q git+https://github.com/HabanaAI/DeepSpeed.git@1.13.0

##### Install the Parameter Efficient Fine Tuning Library methods
This is taking the PEFT method from the Hugging Face repository and will be used to help create the PEFT Fine Tuning with the Llama2 model.

In [4]:
!pip install peft==0.6.2

Cloning into 'peft'...
remote: Enumerating objects: 5894, done.
remote: Counting objects: 100% (1638/1638), done.
remote: Compressing objects: 100% (395/395), done.
remote: Total 5894 (delta 1411), reused 1310 (delta 1225), pack-reused 4256
Receiving objects: 100% (5894/5894), 8.39 MiB | 14.96 MiB/s, done.
Resolving deltas: 100% (3833/3833), done.
/root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference/peft
/root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference


##### Install the Optimum-Habana Library

In [5]:
!pip install -q optimum-habana==1.9.0

##### Pull the Hugging Face Examples from GitHub
These contain the working Hugging Face Task Examples that have been optimized for Gaudi.  For Fine Tuning, we'll use the language-modeling task. 

In [6]:
!git clone https://github.com/huggingface/optimum-habana

Cloning into 'optimum-habana'...
remote: Enumerating objects: 8311, done.
remote: Counting objects: 100% (3188/3188), done.
remote: Compressing objects: 100% (812/812), done.
remote: Total 8311 (delta 2597), reused 2673 (delta 2308), pack-reused 5123
Receiving objects: 100% (8311/8311), 3.83 MiB | 9.24 MiB/s, done.
Resolving deltas: 100% (5521/5521), done.


##### Go to the Language Modeling Task and install the model specific requirements

In [8]:
%cd examples/language-modeling/
!pip install -q -r requirements.txt

/root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference/optimum-habana/examples/language-modeling


##### How to access and Use the Llama 2 model

Use of the pretrained model is subject to compliance with third party licenses, including the “Llama 2 Community License Agreement” (LLAMAV2). For guidance on the intended use of the LLAMA2 model, what will be considered misuse and out-of-scope uses, who are the intended users and additional terms please review and read the instructions in this link https://ai.meta.com/llama/license/.
Users bear sole liability and responsibility to follow and comply with any third party licenses, and Habana Labs disclaims and will bear no liability with respect to users’ use or compliance with third party licenses.

To be able to run gated models like this Llama-2-70b-hf, you need the following: 
- Have a HuggingFace account
- Agree to the terms of use of the model in its model card on the HF Hub
- set a read token
- Login to your account using the HF CLI: run huggingface-cli login before launching your script

In [9]:
!huggingface-cli login --token <your_token_here>

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Fine Tuning the model with PEFT and LoRA

We'll now run the fine tuning with the PEFT method. Remember that the PEFT methods only fine-tune a small number of extra model parameters, thereby greatly decreasing the computational and storage costs. Recent State-of-the-Art PEFT techniques achieve performance comparable to that of full fine-tuning.

##### Here's a summary of the command required to run the Fine Tuning, you'll run this in the next cell below. 
Note in this case the following: 
1. Using the language modeling with LoRA; `run_lora_clm.py`
2. It's very efficient: only 0.02% of the total paramters are being fine tuned of the total 70B parameters.
3. Using DeepSpeed has reduced the max amount of memory to ~73.9 GB out of a total memory available 94.6 GB
4. Only 2 epochs are needed for fine tuning, it takes less than 38 minutes to run with the openassisant-guanaco dataset.  (Running the Llama 2 7B or 13B models will be faster, see the addendum below)

NOTE: In some cases running the full Llama2 70B Fine Tuning in the Juypter Notebook may cause errors.   It's safer to run this in a standard terminal window.   Simply copy the command below and run this in a Terminal window in the same folder location:

In [11]:
!PT_HPU_MAX_COMPOUND_OP_SIZE=10  DEEPSPEED_HPU_ZERO3_SYNC_MARK_STEP_REQUIRED=1 \
python3 ../gaudi_spawn.py --use_deepspeed  --world_size 8 run_lora_clm.py \
  --model_name_or_path meta-llama/Llama-2-70b-hf \
  --deepspeed llama2_ds_zero3_config.json \
  --dataset_name timdettmers/openassistant-guanaco \
  --bf16 True\
  --output_dir ./model_lora_llama \
  --num_train_epochs 2 \
  --max_seq_len 2048 \
  --per_device_train_batch_size 10 \
  --per_device_eval_batch_size 10 \
  --gradient_checkpointing \
  --evaluation_strategy epoch \
  --eval_delay 2 \
  --save_strategy no \
  --learning_rate 0.0018 \
  --warmup_ratio 0.03 \
  --lr_scheduler_type "cosine" \
  --logging_steps 1 \
  --dataset_concatenation \
  --attn_softmax_bf16 True \
  --do_train \
  --do_eval \
  --use_habana \
  --use_lazy_mode \
  --report_to none \
  --pipelining_fwd_bwd \
  --throughput_warmup_steps 3 \
  --lora_rank 4\
  --lora_target_modules "q_proj" "v_proj" "k_proj" "o_proj" \
  --validation_split_percentage 4 

[WARNING|utils.py:198] 2023-12-21 01:48:07,136 >> optimum-habana v1.9.0 has been validated for SynapseAI v1.13.0 but the driver version is v1.12.0, this could lead to undefined behavior!
DistributedRunner run(): command = deepspeed --num_nodes 1 --num_gpus 8 --no_local_rank run_lora_clm.py --model_name_or_path meta-llama/Llama-2-70b-hf --deepspeed llama2_ds_zero3_config.json --dataset_name timdettmers/openassistant-guanaco --bf16 True --output_dir ./model_lora_llama --num_train_epochs 2 --max_seq_len 2048 --per_device_train_batch_size 10 --per_device_eval_batch_size 10 --gradient_checkpointing --evaluation_strategy epoch --eval_delay 2 --save_strategy no --learning_rate 0.0018 --warmup_ratio 0.03 --lr_scheduler_type cosine --logging_steps 1 --dataset_concatenation --attn_softmax_bf16 True --do_train --do_eval --use_habana --use_lazy_mode --report_to none --pipelining_fwd_bwd --throughput_warmup_steps 3 --lora_rank 4 --lora_target_modules q_proj v_proj k_proj o_proj --validation_split_p

#### LoRA Fine Tuning Completed
You will now see a "model_lora_llama" folder created which contains the PEFT model `adapter_model.bin` which will be used in the inference example below. 

## Inference with Llama 2

We'll now use the Hugging Face `text-generation` task to run inference on the Llama2-70b model; we'll generate text based on an included prompt.  Notice that we've included a path to the PEFT model that we just created.

First, we'll move to the text-generation examples folder and install the requirements. 

In [ ]:
%cd /root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference/optimum-habana/examples/text-generation
!pip install -q -r requirements.txt

You will see that we are now running inference with the `run_generation.py` task and we are including the PEFT model that we Fine Tuned in the steps above. 



```
python3 ../gaudi_spawn.py --use_deepspeed  --world_size 8  run_generation.py \
   --model_name_or_path /root/data/llama_inference/Llama-2-70b-chat-hf \
   --batch_size 1 \
   --do_sample \
   --max_new_tokens 250 \
   --n_iterations 4 \
   --use_kv_cache \
   --use_hpu_graphs \
   --bf16 \
   --prompt "I am a dog. Please help me plan a surprise birthday party for my human, including fun activities, games and decorations. And don't forget to order a big bone-shaped cake for me to share with my fur friends!" \
   --peft_model /root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference/optimum-habana/examples/language-modeling/model_lora_llama-chat 
```

In [ ]:
prompt = input("Enter a prompt for text generation: ")

In [ ]:
cmd = f'python3 ../gaudi_spawn.py --use_deepspeed  --world_size 8  run_generation.py \
   --model_name_or_path meta-llama/Llama-2-70b-chat-hf \
   --batch_size 1 --do_sample --max_new_tokens 250 --n_iterations 4 --use_kv_cache --use_hpu_graphs --bf16 --prompt "{prompt}" \
   --peft_model /root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference/optimum-habana/examples/language-modeling/model_lora_llama-chat/'
print(cmd)
import os
os.system(cmd)

###### Inference Output with PEFT

```
input 1: ("I am a dog. Please help me plan a surprise birthday party for my human, including fun activities, games and decorations. And don't forget to order a big bone-shaped cake for me to share with my fur friends!",)
output 1: ('I am a dog. Please help me plan a surprise birthday party for my human, including fun activities, games and decorations. And don\'t forget to order a big bone-shaped cake for me to share with my fur friends!

Assistant: Hey there pup! I can help you plan your human\'s birthday party. Here are some ideas for fun activities and games you can play together:\n\n
1. A "Find the Treat" scavenger hunt: Hide treats around your home or yard for your human to find. Provide clues and hints along the way.\n
2. "Tug-of-War": Play a game of tug-of-war with a rope tied to a tree stump or post.\n
3. "Frisbee Fun": Invite your human to a game of fetch with a Frisbee in the park or backyard.\n\n
Decorations can include: Dog-shaped balloons, paw print streamers, and a banner saying "Happy Birthday" with your human\'s name.\n\n
And don\'t forget to order a cake in the shape of a big bone for you and your fur friends to share!
```

##### Comparison without PEFT and LoRA
In this example, we're simply running the Llama2 7B model **without** including the PEFT fine tuned model, so the you are losing the additional detail that is brought to the model, and the results have signficantly less information and fidelity compared to the last model.

In [ ]:
cmd = f'python3 ../gaudi_spawn.py --use_deepspeed  --world_size 8  run_generation.py \
   --model_name_or_path /root/data/llama_inference/Llama-2-70b-chat-hf \
   --batch_size 1 \
   --do_sample \
   --max_new_tokens 250 \
   --n_iterations 4 \
   --use_kv_cache \
   --use_hpu_graphs \
   --bf16 \
   --prompt "{prompt}"'
print(cmd)
import os
os.system(cmd)

###### Inference Output without PEFT (using just standard Llama 2 model)

```
input 1: ("I am a dog. Please help me plan a surprise birthday party for my human, including fun activities, games and decorations. And don't forget to order a big bone-shaped cake for me to share with my fur friends!",)
output 1: ("I am a dog. Please help me plan a surprise birthday party for my human, including fun activities, games and decorations. And don't forget to order a big bone-shaped cake for me to share with my fur friends!\n

Make sure that you do not make a big noise because my human doesn’t know that we are planning a birthday party. Thanks to your help now I am sure there are no more things to worry about.\n
The dog does not have to worry that the human will find out about the party. She should not worry about the noise while planning the party. There will be big bone-shaped cake for the guest of honor to share with his fur friends. There will be fun activities, games and decorations. The following items are tagged newsletter marketing:\n
```

In [ ]:
exit()

#### ADDENDUM: Running with Llama 2 7B or 13B Parameters
For a faster runtime you can use the smaller Llama 2 7B or 13B model. 

In [ ]:
!python ../gaudi_spawn.py --use_deepspeed \
       --world_size 8 run_lora_clm.py \
       --model_name_or_path meta-llama/Llama-2-7b-hf  \
       --dataset_name timdettmers/openassistant-guanaco \
       --bf16 True \
       --output_dir ./model_lora_llama_7b \
       --num_train_epochs 2 \
       --per_device_train_batch_size 2 \
       --per_device_eval_batch_size 2 \
       --gradient_accumulation_steps 4 \
       --evaluation_strategy "no"\
       --save_strategy "steps"\
       --save_steps 2000 \
       --save_total_limit 1 \
       --learning_rate 1e-4 \
       --logging_steps 1 \
       --dataset_concatenation \
       --do_train \
       --report_to none \
       --use_habana \
       --use_lazy_mode \
       --throughput_warmup_steps 3

Once the Fine Tuning is completed, you can move to the text-generation task to run inference and generate text

In [ ]:
%cd /root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference/optimum-habana/examples/text-generation
!pip install -q -r requirements.txt

In [ ]:
prompt7b = input("Enter a prompt for text generation: ")

This is the output with the PEFT fine tuning added

In [ ]:
cmd = f'python run_generation.py --model_name_or_path meta-llama/Llama-2-7b-hf --batch_size 1 --do_sample --max_new_tokens 250 --n_iterations 4 --use_kv_cache --use_hpu_graphs \
--bf16 --prompt "{prompt7b}" --peft_model /root/Gaudi-tutorials/PyTorch/llama2_fine_tuning_inference/optimum-habana/examples/language-modeling/model_lora_llama_7b/'
print(cmd)
import os
os.system(cmd)

This is the output without the PEFT fine tuning

In [ ]:
cmd = f'python run_generation.py --model_name_or_path meta-llama/Llama-2-7b-hf --batch_size 1 --do_sample --max_new_tokens 250 --n_iterations 4 --use_kv_cache --use_hpu_graphs \
--bf16 --prompt "{prompt7b}" '
print(cmd)
import os
os.system(cmd)

In [ ]:
exit()